Experiment: Anwendung von Azure-Open AI in Verbindung mit Langchain.

Bei der Nutzung von Azure-Open AI ist es erforderlich, sicherheitskritische Informationen wie API-Token zu verwenden. Um deren Sicherheit zu gewährleisten und eine Speicherung in Dateien zu vermeiden, empfiehlt sich die Verwendung von Umgebungsvariablen. Open AI stellt diese bereits vordefiniert bereit.

``` bash
# The API version you want to use: set this to `2023-12-01-preview` for the released version.
export OPENAI_API_VERSION=2024-02-15-preview
# The base URL for your Azure OpenAI resource.  You can find this in the Azure portal under your Azure OpenAI resource.
export AZURE_OPENAI_ENDPOINT=https://your-resource-name.openai.azure.com
# The API key for your Azure OpenAI resource.  You can find this in the Azure portal under your Azure OpenAI resource.
export AZURE_OPENAI_API_KEY=<your Azure OpenAI API key>
``` 

Um die ständige Neueingabe dieser Informationen zu vermeiden, können diese Befehle in die Datei ~/.bashrc eingefügt werden.

Quellen:
- https://python.langchain.com/docs/integrations/llms/azure_openai


In [1]:
import os

from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_deployment = "gpt-4-32k"
)

message = HumanMessage(
    content="Verfasse eine 50 Wörter umfassende Vorlesegeschichte für Kinder, in der es um eine kleine Kartoffel namens Pellemann geht, die einen Hügel hinunterrollt, um ihre Freunde zu suchen."
)
model.invoke([message])

AIMessage(content='Pellemann, die kleine Kartoffel, voller Mut und Tapferkeit, rollte einen großen Hügel hinunter. Er hüpfte und hoppelte, auf der Suche nach seinen Freunden. Oben auf dem Hügel sagte er "Auf Wiedersehen" zu seiner Heimat und lächelte, voll freudiger Erwartung auf das Abenteuer, das vor ihm lag.', response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 61, 'total_tokens': 150}, 'model_name': 'gpt-4-32k', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filter

# Experiment: AzureOpenAI mit Streaming-Function
Manchmal kann die Datenmenge so groß sein, dass die Verarbeitung und Ausgabe der Ergebnisse erhebliche Zeit in Anspruch nimmt. 
Für solche Situationen ist es vorteilhaft, bereits empfangene und verarbeitete Daten fortlaufend auszugeben.

In [2]:
from langchain.callbacks import get_openai_callback
from langchain.callbacks.base import BaseCallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


model = AzureChatOpenAI(
    azure_deployment = "gpt-4",
    streaming=True,
    callback_manager=BaseCallbackManager([
        StreamingStdOutCallbackHandler()]),
)

with get_openai_callback() as cb:
    message = HumanMessage(
        content="Verfasse eine 50 Wörter umfassende Vorlesegeschichte für Kinder, in der es um eine kleine Kartoffel namens Pellemann geht, die einen Hügel hinunterrollt, um ihre Freunde zu suchen."
    )
    model.invoke([message])    

Pellemann, die kleine Kartoffel, rollte fröhlich den Hügel hinunter. Er war auf der Suche nach seinen Freunden. Er rief: "Wo seid ihr?" Da hörte er ein Kichern. Hinter einem Stein, da saßen sie - die anderen Kartoffelkinder. Gemeinsam spielten sie den ganzen Tag und hatten viel Spaß.

# Expereriment AzureOpenAI mit Streaming und Token-Zählung

In Situationen mit umfangreichen Datenmengen kann die Verarbeitungs- und Ausgabezeit der Ergebnisse erheblich sein. 
Um dies zu umgehen, ist es zweckmäßig, die bereits verarbeiteten Daten kontinuierlich auszugeben. 
Darüber hinaus beinhaltet dieses Experiment eine Funktion, die die verarbeiteten Tokens zählt und diese Information ebenfalls ausgibt.
Dies wird durch die Vererbung der BaseCallbackHandler-Klasse ermöglicht.

In [3]:
from langchain.callbacks import get_openai_callback
from langchain.callbacks.base import BaseCallbackManager, BaseCallbackHandler
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.outputs.llm_result import LLMResult

class MyCustomHandler(BaseCallbackHandler):
    tokenCount = 0  

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        self.tokenCount += 1
        print(token, end="")

    def on_llm_end(self, response: LLMResult, **kwargs) -> None:        
        print(f"\nAnzahl token: {self.tokenCount}")

model = AzureChatOpenAI(
    azure_deployment = "gpt-4",
    streaming=True,
    callback_manager=BaseCallbackManager([
        MyCustomHandler()]),
)

with get_openai_callback() as cb:
    message = HumanMessage(
        content="Verfasse eine 50 Wörter umfassende Vorlesegeschichte für Kinder, in der es um eine kleine Kartoffel namens Pellemann geht, die einen Hügel hinunterrollt, um ihre Freunde zu suchen."
    )
    model.invoke([message])    

Einmal lebte eine kleine Kartoffel namens Pellemann. Eines Tages rollte er einen Hügel hinunter, auf der Suche nach seinen Freunden. Er hüpfte über Steine, glitt durch Gras und lachte dabei fröhlich. Am Fuße des Hügels fand er schließlich seine Freunde und spielte den ganzen Tag glücklich mit ihnen.
Anzahl token: 91
